In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df=pd.read_csv('/content/drive/My Drive/synthetic_logs.csv')
df.head()


,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert


In [4]:
df = df.drop('complexity', axis=1)
df=df.drop('timestamp', axis=1)
display(df.head())

,source,log_message,target_label
0,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status
1,ModernCRM,Email service experiencing issues with sending,Critical Error
2,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert
3,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status
4,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status


In [5]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [6]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

In [7]:
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('all-MiniLM-L6-v2')
embeddings=model.encode(df['log_message'].tolist())
embeddings[:2]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


array([[-1.02939598e-01,  3.35459486e-02, -2.20260844e-02,
         1.55103172e-03, -9.86921880e-03, -1.78956211e-01,
        -6.34409934e-02, -6.01761453e-02,  2.81108953e-02,
         5.99620081e-02, -1.72618236e-02,  1.43364200e-03,
        -1.49560079e-01,  3.15288268e-03, -5.66030741e-02,
         2.71685328e-02, -1.49890278e-02, -3.54037210e-02,
        -3.62936370e-02, -1.45410486e-02, -5.61492983e-03,
         8.75538811e-02,  4.55120727e-02,  2.50963680e-02,
         1.00187613e-02,  1.24267004e-02, -1.39923558e-01,
         7.68696666e-02,  3.14095393e-02, -4.15247958e-03,
         4.36902344e-02,  1.71249956e-02, -8.00950900e-02,
         5.74006140e-02,  1.89092122e-02,  8.55262056e-02,
         3.96399088e-02, -1.34371832e-01, -1.44367013e-03,
         3.06707830e-03,  1.76854059e-01,  4.44890792e-03,
        -1.69275142e-02,  2.24266183e-02, -4.35049757e-02,
         6.09031972e-03, -9.98171885e-03, -6.23973012e-02,
         1.07372692e-02, -6.04895223e-03, -7.14661255e-0

In [8]:
dbscan=DBSCAN(eps=0.2,min_samples=1,metric='cosine')
clusters=dbscan.fit_predict(embeddings)
df['cluster']=clusters
df.head()

,source,log_message,target_label,cluster
0,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,0
1,ModernCRM,Email service experiencing issues with sending,Critical Error,1
2,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,2
3,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,0
4,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,0


In [9]:
df[df.cluster==1]

,source,log_message,target_label,cluster
1,ModernCRM,Email service experiencing issues with sending,Critical Error,1
10,ModernCRM,Email server encountered a sending fault,Error,1
217,BillingSystem,Mail service encountered a delivery glitch,Error,1
248,ModernHR,Service disruption caused by email sending error,Critical Error,1
265,ModernCRM,Email system had a problem sending emails,Error,1
361,BillingSystem,Email service experienced a sending issue,Error,1
450,ThirdPartyAPI,Email delivery system encountered an error,Error,1
477,AnalyticsEngine,Email transmission error caused service impact,Critical Error,1
570,ThirdPartyAPI,Email service impacted by sending failure,Critical Error,1
678,AnalyticsEngine,Email delivery problem affected system,Critical Error,1


In [10]:
cluster_counts = df['cluster'].value_counts()
large_clusters = cluster_counts[cluster_counts > 10].index.tolist()

for cluster_id in large_clusters:
    print(f"Cluster {cluster_id}:")
    cluster_messages = df[df['cluster'] == cluster_id]['log_message'].sample(min(5, len(df[df['cluster'] == cluster_id])))
    for message in cluster_messages:
        print(f"- {message}")
    print("\n")

Cluster 0:
- nova.osapi_compute.wsgi.server [req-05bac4af-55c4-4d90-bd9e-f782858bd4d1 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" RCODE  200 len: 1910 time: 0.2505140
- nova.osapi_compute.wsgi.server [req-c58e5c9b-8211-4631-a4dc-8be93e96a0d2 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" Status code -  200 len: 1893 time: 0.2586210
- nova.osapi_compute.wsgi.server [req-97a94302-7980-4793-9333-2254342e1d40 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2462721
- nova.osapi_compute.wsgi.server [req-68b928fe-2e4c-41b1-b891-baef5468e91f 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed933

In [11]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out)": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return label
    return None

In [14]:
classify_with_regex("User User564 Logged in")

'User Action'

In [15]:
df['regex_label']=df['log_message'].apply(classify_with_regex)
df_non_regex=df[df.regex_label.isna()]
df_non_regex.head(20)

,source,log_message,target_label,cluster,regex_label
0,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,0,None
1,ModernCRM,Email service experiencing issues with sending,Critical Error,1,None
2,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,2,None
3,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,0,None
4,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,0,None
5,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,0,None
6,ModernHR,Shard 6 replication task ended in failure,Error,3,None
8,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,5,None
9,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,0,None
10,ModernCRM,Email server encountered a sending fault,Error,1,None


In [17]:
df_without_legacy=df_non_regex[df_non_regex.source!='LegacyCRM']
df_without_legacy.head()


,source,log_message,target_label,cluster,regex_label
0,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,0,None
1,ModernCRM,Email service experiencing issues with sending,Critical Error,1,None
2,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,2,None
3,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,0,None
4,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,0,None


In [ ]:
df_without_legacy.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI'], dtype=object)

In [18]:
#this embedding is without the Legacy CRM
new_embeddings=model.encode(df_without_legacy['log_message'].tolist())
new_embeddings[:2]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


array([[-1.02939598e-01,  3.35459486e-02, -2.20260844e-02,
         1.55103172e-03, -9.86921880e-03, -1.78956211e-01,
        -6.34409934e-02, -6.01761453e-02,  2.81108953e-02,
         5.99620081e-02, -1.72618236e-02,  1.43364200e-03,
        -1.49560079e-01,  3.15288268e-03, -5.66030741e-02,
         2.71685328e-02, -1.49890278e-02, -3.54037210e-02,
        -3.62936370e-02, -1.45410486e-02, -5.61492983e-03,
         8.75538811e-02,  4.55120727e-02,  2.50963680e-02,
         1.00187613e-02,  1.24267004e-02, -1.39923558e-01,
         7.68696666e-02,  3.14095393e-02, -4.15247958e-03,
         4.36902344e-02,  1.71249956e-02, -8.00950900e-02,
         5.74006140e-02,  1.89092122e-02,  8.55262056e-02,
         3.96399088e-02, -1.34371832e-01, -1.44367013e-03,
         3.06707830e-03,  1.76854059e-01,  4.44890792e-03,
        -1.69275142e-02,  2.24266183e-02, -4.35049757e-02,
         6.09031972e-03, -9.98171885e-03, -6.23973012e-02,
         1.07372692e-02, -6.04895223e-03, -7.14661255e-0

**Reasoning**:
Split the data into training and testing sets for logistic regression.



In [19]:
from sklearn.model_selection import train_test_split

X = new_embeddings
y = df_without_legacy['target_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

**Reasoning**:
Train a Logistic Regression model on the training data.



In [20]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [21]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

                precision    recall  f1-score   support

Critical Error       0.91      1.00      0.95        48
         Error       0.98      0.89      0.93        47
   HTTP Status       1.00      1.00      1.00       304
Resource Usage       1.00      1.00      1.00        49
Security Alert       1.00      0.99      1.00       123

      accuracy                           0.99       571
     macro avg       0.98      0.98      0.98       571
  weighted avg       0.99      0.99      0.99       571



In [27]:
import joblib

# Dump the model to a file
joblib.dump(model, 'log_classifier.pkl')

['log_classifier.pkl']

In [42]:
import shutil

# Define the source and destination paths
source_path = 'log_classifier.pkl'
destination_path = '/content/drive/My Drive/log_classifier.pkl'

# Copy the file to Google Drive
shutil.copy(source_path, destination_path)

print(f"Model saved to {destination_path}")

Model saved to /content/drive/My Drive/log_classifier.pkl


System Notification
User Action
None


In [6]:
import re
import joblib
from sentence_transformers import SentenceTransformer
from google.colab import userdata
from groq import Groq

# Define regex patterns
regex_patterns = {
    r"User User\d+ logged (in|out).": "User Action",
    r"Backup (started|ended) at .*": "System Notification",
    r"Backup completed successfully.": "System Notification",
    r"System updated to version .*": "System Notification",
    r"File .* uploaded successfully by user .*": "System Notification",
    r"Disk cleanup completed successfully.": "System Notification",
    r"System reboot initiated by user .*": "System Notification",
    r"Account with ID .* created by .*": "User Action"
}

# Load the BERT model and the trained classifier model
try:
    bert_model = SentenceTransformer('all-MiniLM-L6-v2')
    classifier_model = joblib.load('/content/drive/My Drive/logistic_regression_model.pkl') # Load from Drive
except FileNotFoundError:
    print("Error loading BERT or classifier model. Please ensure the files exist.")
    bert_model = None
    classifier_model = None


# Initialize Groq client
groq_api_key = userdata.get('GROQ_TOKEN') # Ensure 'GROQ_TOKEN' is the correct secret name
if groq_api_key is None:
  print("Error: GROQ_TOKEN not found in Colab secrets. Please add it to the Secrets Manager.")
  llm_client = None
else:
  llm_client = Groq(api_key=groq_api_key)


def classify_with_regex(log_message):
    """Classifies a log message using regex patterns."""
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return label
    return None

def classify_with_bert(log_message):
    """Classifies a log message using the BERT model and the trained classifier."""
    if bert_model and classifier_model:
        try:
            embedding = bert_model.encode([log_message])
            probabilities = classifier_model.predict_proba(embedding)[0]
            if max(probabilities) < 0.5:
                return "unclassified"
            predicted = classifier_model.predict(embedding)[0]
            return predicted
        except Exception as e:
            print(f"Error during BERT classification: {e}")
            return "unclassified" # Return unclassified on error
    else:
        print("BERT or classifier model not loaded.")
        return "unclassified" # Return unclassified if models are not loaded

def classify_llm(log_message):
    """Classifies a log message using the LLM."""
    if llm_client:
        prompt_text=f'''Classify the log message into one of these categories:
        (1) WorkFlow Error (2) Depreciation Warning (3) Unclassified.
        Only return the Category Name. No Preamble
        Log Message: {log_message} ''' # Added Unclassified to the prompt

        try:
            completion = llm_client.chat.completions.create(
                model="llama-3.3-70b-versatile", # Using a more commonly available Llama model
                messages=[
                  {
                    "role": "user",
                    "content": prompt_text,
                  }
                ]
            )
            # Process LLM response to extract the category name
            response_text = completion.choices[0].message.content.strip()
            # Simple check for known categories, otherwise return Unclassified
            if response_text in ["WorkFlow Error", "Depreciation Warning", "Unclassified"]:
                return response_text
            else:
                 # If LLM returns something unexpected, still return Unclassified
                 return "Unclassified"
        except Exception as e:
            print(f"An error occurred during LLM API call: {e}")
            return "Unclassified" # Return Unclassified on API error
    else:
        print("LLM client not initialized.")
        return "Unclassified" # Return Unclassified if client is not initialized

def classify_log(source, log_msg):
    """Classifies a single log message based on source and content."""
    if source == "LegacyCRM":
        # Use LLM for LegacyCRM logs
        label = classify_llm(log_msg)
        return label
    else:
        # Try regex first for other sources
        label = classify_with_regex(log_msg) # Added log_msg as argument
        if label:
            return label
        else:
            # If regex doesn't classify, use BERT model
            label = classify_with_bert(log_msg)
            return label

def classify(logs):
  """Classifies a list of log messages."""
  labels=[]
  for source,log_msg in logs:
    label=classify_log(source,log_msg)
    labels.append(label)
  return labels

def classify_csv(input_file):
    import pandas as pd
    df = pd.read_csv(input_file)

    df = pd.read_csv(input_file)
    df["target_label"] = classify(list(zip(df["source"], df["log_message"])))
    return df
if __name__ == "__main__":
    df_result=classify_csv('/content/drive/My Drive/test.csv')
    print(df_result.head())





/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


            source                                        log_message  \
0        ModernCRM   "IP 192.168.133.114 blocked due to potential ...   
1    BillingSystem                        "User User12345 logged in."   
2  AnalyticsEngine   "File data_6957.csv uploaded successfully by ...   
3  AnalyticsEngine                   "Backup completed successfully."   
4         ModernHR   "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/ser...   

          target_label  
0       Security Alert  
1          User Action  
2  System Notification  
3  System Notification  
4          HTTP Status  


In [140]:
!pip install --upgrade transformers sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 784.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal